# Ансамбль классификации

1. Сформировать параллельный ансамбль из логистической регрессии, SVM, случайного леса и LightGBM
2. Использовать лучшие гиперпараметры, подобранные ранее
3. Сделать предсказание и проверить качество через каппа-метрику

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn import preprocessing

from core.reduce_mem_usage import reduce_mem_usage

## Загрузка данных

In [2]:
data = pd.read_csv("../data/prudential/train.csv.gz")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


## Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)

In [4]:
list_df = [data]
for un in data["Product_Info_2_1"].unique():
    s = pd.DataFrame()
    s["Product_Info_2_1_" + un] = data["Product_Info_2_1"].isin([un]).astype("int8")
    list_df.append(s)
data = pd.concat(list_df, axis=1)
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)

In [5]:
data.fillna(value=-1, inplace=True)
data["Response"] = data["Response"] - 1
print(data.head())

   Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   2               1              10        0.076923               2   
1   5               1              26        0.076923               2   
2   6               1              26        0.076923               2   
3   7               1              10        0.487179               2   
4   8               1              26        0.230769               2   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0               1               1  0.641791  0.581818  0.148536  ...   
1               3               1  0.059701  0.600000  0.131799  ...   
2               3               1  0.029851  0.745455  0.288703  ...   
3               3               1  0.164179  0.672727  0.205021  ...   
4               3               1  0.417910  0.654545  0.234310  ...   

   Medical_Keyword_46  Medical_Keyword_47  Medical_Keyword_48  Response  \
0                   0                   0            

## Набор столбцов для расчета

In [6]:
columns_groups = [
    "Insurance_History",
    "InsuredInfo",
    "Medical_Keyword",
    "Family_Hist",
    "Medical_History",
    "Product_Info",
]
columns = [
    "Wt",
    "Ht",
    "Ins_Age",
    "BMI",
]

for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print(columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keywor

## Нормализация данных

In [7]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data, columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

In [8]:
data_transformed = reduce_mem_usage(data_transformed)
print(data_transformed.info())

Потребление памяти меньше на - 42.87 Мб (минус 75.1%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 126 entries, 0 to Response
dtypes: float16(125), int8(1)
memory usage: 14.2 MB
None


## Разделить данные

In [9]:
data_train, data_test = train_test_split(data_transformed, test_size=0.2)
print(data_train.head())

              0         1         2         3         4         5         6  \
46300  0.825684 -1.200195 -0.467285  2.105469  0.611816 -0.169434 -1.159180   
37302  4.257812  1.249023 -0.467285  3.933594  0.611816 -0.169434 -1.159180   
230   -1.571289 -0.220581 -0.391602 -1.843750  0.611816 -0.169434 -1.159180   
18358  0.426270  1.003906 -0.845703 -0.081970  0.611816 -0.169434  0.862305   
21506 -0.983398 -0.220581  0.062622 -1.086914  0.611816 -0.169434  0.862305   

              7         8         9  ...       116       117       118  \
46300  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
37302  1.100586 -1.156250  1.130859  ... -0.083679 -2.263672 -0.149292   
230    1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
18358 -1.013672  0.880859 -0.928711  ... -0.083679  0.441650 -0.149292   
21506  0.043671  0.863770 -0.928711  ... -0.083679  0.441650 -0.149292   

            119       120       121       122       123      124  Response  
463

## Построение базовых моделей

In [ ]:
x = pd.DataFrame(data_train, columns=columns_transformed)

### SVM

In [ ]:
model_svm = SVC(kernel='linear', probability=True, max_iter=10000)
model_svm.fit(x, data_train["Response"])

### Логистическая регрессия

In [ ]:
model_logr = LogisticRegression(max_iter=1000)
model_logr.fit(x, data_train["Response"])

### Случайный лес

In [ ]:
model_rf = RandomForestClassifier(
    random_state=17,
    n_estimators=76,
    max_depth=17,
    max_features=27,
    min_samples_leaf=20
)
model_rf.fit(x, data_train["Response"])

### LightGBM, используем multiclass

In [13]:
model_lgb = lgb.LGBMRegressor(
    random_state=17,
    max_depth=18,
    min_child_samples=18,
    num_leaves=75,
    n_estimators=1000,
    objective="multiclass",
    num_class=8
)
model_lgb.fit(x, data_train["Response"])

## Предсказание данных

In [ ]:
def calculate_model(x):
    return np.argmax(model.predict([x]))

In [ ]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)

In [15]:
data_test_svm_proba = pd.DataFrame(model_svm.predict_proba(x_test))

In [ ]:
data_test_logr_proba = pd.DataFrame(model_logr.predict_proba(x_test))

In [ ]:
data_test_rf_proba = pd.DataFrame(model_rf.predict_proba(x_test))

In [ ]:
data_test_lgb = pd.DataFrame(model_lgb.predict(x_test))

In [ ]:
def vote_class(x):
    a = np.argmax(x.values)
    return a

In [ ]:
data_test_proba = data_test_svm_proba.copy()
for i in range(0, 8):
    data_test_proba[i] = 5*data_test_proba[i] 
    data_test_proba[i] = data_test_proba[i] + 5*data_test_logr_proba[i]
    data_test_proba[i] = data_test_proba[i] + data_test_rf_proba[i]
    data_test_proba[i] = data_test_proba[i] + 12*data_test_lgb[i]
data_test_proba["target"] = data_test_proba.apply(vote_class, axis=1)

## Оценка модели

In [ ]:
print('Ансамбль классификации:'
      round(cohen_kappa_score(data_test_proba["target"], data_test["Response"], weights="quadratic"), 3))

## Матрица неточностей

In [ ]:
print(f'Ансамбль классификации:\n\n {confusion_matrix(data_test_proba["target"], data_test["Response"])}')

## Самопроверка

In [ ]:
data_copy = data_train.copy()
x_copy = pd.DataFrame(data_copy, columns=columns_transformed)

In [ ]:
data_copy_svm = pd.DataFrame(model_svm.predict_proba(x_copy))

In [ ]:
data_copy_logr = pd.DataFrame(model_logr.predict_proba(x_copy))

In [ ]:
data_copy_rf = pd.DataFrame(model_rf.predict_proba(x_copy))

In [ ]:
data_copy_lgb = pd.DataFrame(model_lgb.predict(x_copy))

In [ ]:
for i in range(0, 8):
    data_copy_svm[i] = 5*data_copy_svm[i] 
    data_copy_svm[i] = data_copy_svm[i] + 5*data_copy_logr[i]
    data_copy_svm[i] = data_copy_svm[i] + data_copy_rf[i]
    data_copy_svm[i] = data_copy_svm[i] + 12*data_copy_lgb[i]
target = data_copy_svm.apply(vote_class, axis=1)

In [ ]:
print('Результат:'
      round(cohen_kappa_score(target, data_copy["Response"], weights="quadratic"), 3))

In [ ]:
print(confusion_matrix(target, data_copy["Response"]))